### Homework: Agents

In this homework, we will learn more about function calling, and we will also explore MCP - model-context-protocol.

### Preparation

First, we'll define a function that we will use when building our agent.
It will generate gake weather data:

In [3]:
import random

known_weather_data = {
    'berlin': 20.0
}

def get_weather(city: str) -> float:
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)

### Q1. Define function description

We want to use it as a tool for our agent, so we need to describe it

How should the description for this function look like? Fill in missing parts

In [4]:
get_weather_tool = {
    "type": "function",
    "name": "get_weather",
    "description": "Get City Weather",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "Gets the current weather from the selected city"
            }
        },
        "required": ["city"],
        "additionalProperties": False
    }
}

What did you put in TODO3?

### A1

`'city'`

### Testing it (Optional)
If you have OpenAI API Key (or alternative provider), let's test it.

A question could be "What's the weather like in Germany?"

Experiment with different system prompts to have better answers from the system.

You can use chat_assistant.py or implement everything yourself

In [7]:
!wget https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py

--2025-07-14 20:39:41--  https://raw.githubusercontent.com/alexeygrigorev/rag-agents-workshop/refs/heads/main/chat_assistant.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3485 (3.4K) [text/plain]
Saving to: ‘chat_assistant.py’

chat_assistant.py   100%[===================>]   3.40K  --.-KB/s    in 0s      

2025-07-14 20:39:41 (30.4 MB/s) - ‘chat_assistant.py’ saved [3485/3485]



In [8]:
from openai import OpenAI
client = OpenAI()

def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content

In [9]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)

tools.get_tools()

developer_prompt = """
You're a assistant.
You're a given a question about the weather and your task is to answer it
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [10]:
chat.run()

You: how's the weather in Berlin


You: stop


Chat ended.


### Q2. Adding another tool

Let's add another tool - a function that can add weather data to our database:

```bash 
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'
```

Now let's write a description for it.

What did you write?

Optionally, you can test it after adding this function.

In [11]:
def set_weather(city: str, temp: float) -> None:
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

In [12]:
add_weather_temp = {
    "type": "function",
    "name": "set_weather",
    "description": "Add an entry to the weather database",
    "parameters": {
        "type": "object",
        "properties": {
            "city": {
                "type": "string",
                "description": "The weather information to be added to the city weather database",
            },
            "temp": {
                "type": "string",
                "description": "The weather temperature",
            }
        },
        "required": ["city", "temp"],
        "additionalProperties": False
    }
}

In [13]:
import chat_assistant

tools = chat_assistant.Tools()
tools.add_tool(get_weather, get_weather_tool)


In [14]:
tools.add_tool(set_weather, add_weather_temp)

In [15]:
tools.get_tools()

[{'type': 'function',
  'name': 'get_weather',
  'description': 'Get City Weather',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'Gets the current weather from the selected city'}},
   'required': ['city'],
   'additionalProperties': False}},
 {'type': 'function',
  'name': 'set_weather',
  'description': 'Add an entry to the weather database',
  'parameters': {'type': 'object',
   'properties': {'city': {'type': 'string',
     'description': 'The weather information to be added to the city weather database'},
    'temp': {'type': 'string', 'description': 'The weather temperature'}},
   'required': ['city', 'temp'],
   'additionalProperties': False}}]

In [16]:
developer_prompt = """
You're a assistant.
You're a given a question about the weather and your task is to answer it
""".strip()

chat_interface = chat_assistant.ChatInterface()

chat = chat_assistant.ChatAssistant(
    tools=tools,
    developer_prompt=developer_prompt,
    chat_interface=chat_interface,
    client=client
)

In [17]:
chat.run()

You: how's the weather in vancouver


You: Add the result into the database


You: stop


Chat ended.


### MCP

MCP stands for Model-Context Protocol. It allows LLMs communicate with different tools (like Qdrant). It's function calling, but one step further:

* A tool can export a list of functions it has
* When we include the tool to our Agent, we just need to include the link to the MCP server

### Q3. Install FastMCP

In [22]:
!pip install fastmcp


[notice] A new release of pip is available: 24.0 -> 25.1.1
[notice] To update, run: pip install --upgrade pip


### A3.

What's the version of FastMCP you installed?

In [25]:
import fastmcp
fastmcp.__version__

'2.10.5'

### Q4. Simple MCP Server

A simple MCP server from the documentation looks like that:

# weather_server.py
```bash 
from fastmcp import FastMCP

mcp = FastMCP("Demo 🚀")

@mcp.tool
def add(a: int, b: int) -> int:
    """Add two numbers"""
    return a + b

if __name__ == "__main__":
    mcp.run()
```

In our case, we need to write docstrings for our functions.

Let's ask ChatGPT for help:

```bash

def get_weather(city: str) -> float:
    """
    Retrieves the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to retrieve weather data.

    Returns:
        float: The temperature associated with the city.
    """
    city = city.strip().lower()

    if city in known_weather_data:
        return known_weather_data[city]

    return round(random.uniform(-5, 35), 1)


def set_weather(city: str, temp: float) -> None:
    """
    Sets the temperature for a specified city.

    Parameters:
        city (str): The name of the city for which to set the weather data.
        temp (float): The temperature to associate with the city.

    Returns:
        str: A confirmation string 'OK' indicating successful update.
    """
    city = city.strip().lower()
    known_weather_data[city] = temp
    return 'OK'

```

Let's change the example for our case and run it

What do you see in the output?

Look for a string that matches this template:

```bash
Starting MCP server 'Demo 🚀' with transport '<TODO>'
```

What do you have instead of `<TODO>?` 

### A4.

```bash
╭─ FastMCP 2.0 ──────────────────────────────────────────────────────────────╮
│                                                                            │
│        _ __ ___ ______           __  __  _____________    ____    ____     │
│       _ __ ___ / ____/___ ______/ /_/  |/  / ____/ __ \  |___ \  / __ \    │
│      _ __ ___ / /_  / __ `/ ___/ __/ /|_/ / /   / /_/ /  ___/ / / / / /    │
│     _ __ ___ / __/ / /_/ (__  ) /_/ /  / / /___/ ____/  /  __/_/ /_/ /     │
│    _ __ ___ /_/    \__,_/____/\__/_/  /_/\____/_/      /_____(_)____/      │
│                                                                            │
│                                                                            │
│                                                                            │
│    🖥️  Server name:     Demo 🚀                                             │
│    📦 Transport:       STDIO                                               │
│                                                                            │
│    📚 Docs:            https://gofastmcp.com                               │
│    🚀 Deploy:          https://fastmcp.cloud                               │
│                                                                            │
│    🏎️  FastMCP version: 2.10.5                                              │
│    🤝 MCP version:     1.11.0                                              │
│                                                                            │
╰────────────────────────────────────────────────────────────────────────────╯


[07/14/25 23:56:30] INFO     Starting MCP server 'Demo 🚀' with transport 'stdio'    
```

### Q5. Protocol

There are different ways to communicate with an MCP server. Ours is currently running using standart input/output, which means that the client write something to stdin and read the answer using stdout.

Our weather server is currently running.

This is how we start communitcating with it:

First, we send an initialization request -- this way, we register our client with the server

```bash
{"jsonrpc": "2.0", "id": 1, "method": "initialize", "params": {"protocolVersion": "2024-11-05", "capabilities": {"roots": {"listChanged": true}, "sampling": {}}, "clientInfo": {"name": "test-client", "version": "1.0.0"}}}
```

We should get back something like that, which is an aknowledgement of the request:

```bash
{"jsonrpc":"2.0","id":1,"result":{"protocolVersion":"2024-11-05","capabilities":{"experimental":{},"prompts":{"listChanged":false},"resources":{"subscribe":false,"listChanged":false},"tools":{"listChanged":true}},"serverInfo":{"name":"Demo 🚀","version":"1.9.4"}}}
```

* Next, we reply back, confirming the initialization:
```bash
{"jsonrpc": "2.0", "method": "notifications/initialized"}
```

We don't expect to get anything in response
* Now we can ask for a list of available methods
```bash
{"jsonrpc": "2.0", "id": 2, "method": "tools/list"}
```

* Let's ask the temperature in Berlin:
```bash
{"jsonrpc": "2.0", "id": 3, "method": "tools/call", "params": {"name": "<TODO>", "arguments": {<TODO>}}}
```

What did you get in response?

### A5.

```bash
{"jsonrpc":"2.0","id":3,"result":{"content":[{"type":"text","text":"20.0"}],"structuredContent":{"result":20.0},"isError":false}}
```

### Q6. Client

We typically don't interact with the server by copy-pasting commands in the terminal.

In practice, we use an MCP Client. Let's implement it.

FastMCP also supports MCP clients:

```bash
from fastmcp import Client

async def main():
    async with Client(<TODO>) as mcp_client:
        # TODO
```

Use the client to get the list of available tools of our script. How does the result look like?

If you're running this code in Jupyter, you need to pass an instance of MCP server to the Client:

```bash
import weather_server

async def main():
    async with Client(weather_server.mcp) as mcp_client:
        # ....
```

If you run it in a script, you will need to use asyncio:

```bash
import asyncio

async def main():
    async with Client("weather_server.py") as mcp_client:
        # ...

if __name__ == "__main__":
    test = asyncio.run(main())
```

Copy the output with the available tools when filling in the homework form.

### A6.

In [28]:
import asyncio
from fastmcp import Client
import mcp_file  # Only if you're importing mcp from another file

async def main():
    async with Client(mcp_file.mcp) as mcp_client:  # or just `mcp` if already in scope
        tools = await mcp_client.list_tools()
        print("Available tools:")
        for tool in tools:
            print(f"- {tool.name}: {tool.description}")

# In Jupyter, you run this with:
await main()

Available tools:
- add: Add two numbers
- get_weather: Retrieves the temperature for a specified city.

Parameters:
    city (str): The name of the city for which to retrieve weather data.

Returns:
    float: The temperature associated with the city.
- set_weather: Sets the temperature for a specified city.

Parameters:
    city (str): The name of the city for which to set the weather data.
    temp (float): The temperature to associate with the city.

Returns:
    str: A confirmation string 'OK' indicating successful update.
